In [51]:
import sys
import importlib.util

# Specify the path to the module
module_path = "/Users/manojkl/Documents/ENV/env.py"

# Load the module dynamically
spec = importlib.util.spec_from_file_location("env", module_path)
my_module = importlib.util.module_from_spec(spec)

# Add the module to sys.modules so it can be used
sys.modules["env"] = my_module

# Load the module
spec.loader.exec_module(my_module)

api_key = my_module.OPENROUTER_API_KEY

In [52]:
from openai import OpenAI


def gemini_flash(content):
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=api_key,
    )

    # Define the model and the content
    model_id = "google/gemini-2.0-flash-thinking-exp:free"
    # content = "Write a Python function to calculate Fibonacci numbers recursively."

    # Generate content using the Thinking Mode model
    response = client.chat.completions.create(
        model=model_id, messages=[{"role": "user", "content": content}]
    )

    if response.choices and response.choices[0].message:
        output = response.choices[0].message.content
        # print(output)
        # print(type(content))
    else:
        output = f"Solution: {content}\nCritique:"
        print("No message content found.")

    return output

In [53]:
def meta(content):
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=api_key,
    )

    response = client.chat.completions.create(
        model="meta-llama/llama-3.2-3b-instruct:free",
        messages=[{"role": "user", "content": content}],
    )
    # print(completion.choices[0].message.content)
    if response.choices and response.choices[0].message:
        output = response.choices[0].message.content
        # print(output)
        # print(type(content))
    else:
        output = f"Solution: {content}\nCritique:"
        print("No message content found.")

    return output

# meta("Write a Python function to calculate Fibonacci numbers recursively.")

In [54]:
def write_to_file(input_string):
    # Open a text file in write mode
    with open('output.txt', 'a') as file:
        # Write the input string to the file
        file.write(input_string + '\n')
        # Append "================" at the end
        file.write("=================================================================================\n")

In [ ]:
def iterative_refinement(problem, max_iterations=5):
    iteration = 0
    solution = problem

    while iteration < max_iterations:
        print(f"\n--- Iteration {iteration + 1} ---")
        write_to_file(f"\n--- Iteration {iteration + 1} ---")
        # LLM-A generates a response
        solution = gemini_flash(f"IMPROTANT: Python Code is must. Task: {problem}\nSolution:")
        write_to_file(solution)
        # print(f"Gemini Flash's Response:\n{solution}")

        # # LLM-B critiques the response
        critique = meta(
            f"IMPROTANT: Python Code is must. Solution: {solution}\nCritique:"
        )
        # # print(f"Meta Critique:\n{critique}")
        write_to_file(critique)

        # # Evaluate stopping condition (can refine further logic here)
        if "no issues" in critique.lower():
            print("Consensus reached.")
            break

        # # Prepare input for next iteration
        problem = critique  # Alternatively, merge critique with original task
        iteration += 1

    return solution

# Example usage
problem = "Write a Python function to calculate Fibonacci numbers recursively."
final_solution = iterative_refinement(problem)
print(f"\nFinal Solution:\n{final_solution}")


--- Iteration 1 ---

--- Iteration 2 ---

--- Iteration 3 ---

--- Iteration 4 ---

--- Iteration 5 ---

Final Solution:
My thinking process to generate the improved explanation of relative motion went through several stages, focusing on addressing the feedback points systematically and incorporating best practices for effective explanation.

1. **Deconstructing the Feedback:** I first broke down the feedback into actionable points.  I identified the strengths to maintain and the weaknesses to address.  The key areas for improvement were:
    * **Precise Language:**  Show *how* to use terms better.
    * **More Detail in Examples:**  Expand on examples for deeper understanding.
    * **Transitional Phrases:** Improve flow and coherence.
    * **Emphasis on the Core Concept:** Differentiate from absolute motion.
    * **Adding a Conclusion:**  Summarize and reinforce.

2. **Planning the Structure:** I decided to maintain the existing well-organized structure identified as a strength.  